In [1]:
import pandas as pd
import numpy as np

In [2]:
TRAIN_DATA = "data/preprocessed/train_data.csv"

In [3]:
train_df = pd.read_csv(TRAIN_DATA)

In [4]:
train_df.shape

(20974, 41)

In [6]:
train_df.isnull().sum()

Price                     0
city                      0
Area                      0
Location                808
No. of Bedrooms           0
Resale                  962
MaintenanceStaff       1271
Gymnasium              1063
SwimmingPool           1101
LandscapedGardens      1026
JoggingTrack            857
RainWaterHarvesting     858
IndoorGames             761
ShoppingMall            816
Intercom               1014
SportsFacility         1008
ATM                    1103
ClubHouse              1115
School                    0
24X7Security              0
PowerBackup               0
CarParking              892
StaffQuarter              0
Cafeteria               628
MultipurposeRoom        779
Hospital               1136
WashingMachine          767
Gasconnection           911
AC                      734
Wifi                   1344
Children'splayarea      775
LiftAvailable             0
BED                       0
VaastuCompliant         988
Microwave              1441
GolfCourse          

In [7]:
train_df.head()

,Price,city,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,...,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Stadium
0,45000000,Kolkata,900,Murali,2,1.0,9.0,9.0,9.0,9.0,...,9,9,9.0,9.0,9.0,9.0,9,9,9,9.0
1,7504000,Bangalore,1204,Kalyan Nagar,2,0.0,9.0,9.0,9.0,NaN,...,9,9,9.0,9.0,9.0,9.0,9,9,9,9.0
2,18500000,Mumbai,1150,Powai,2,1.0,9.0,9.0,9.0,9.0,...,9,9,9.0,9.0,9.0,9.0,9,9,9,NaN
3,6500000,Kolkata,1359,Sodepur,3,0.0,NaN,9.0,9.0,9.0,...,9,9,9.0,NaN,9.0,NaN,9,9,9,9.0
4,5800000,Mumbai,1000,Kamothe,2,1.0,1.0,1.0,0.0,0.0,...,1,1,0.0,0.0,0.0,0.0,0,0,0,0.0


In [8]:
train_df['price_per_foot^2'] = train_df['Price']/train_df['Area']

In [9]:
train_df.head()

,Price,city,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,...,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Stadium,price_per_foot^2
0,45000000,Kolkata,900,Murali,2,1.0,9.0,9.0,9.0,9.0,...,9,9.0,9.0,9.0,9.0,9,9,9,9.0,50000.000000
1,7504000,Bangalore,1204,Kalyan Nagar,2,0.0,9.0,9.0,9.0,NaN,...,9,9.0,9.0,9.0,9.0,9,9,9,9.0,6232.558140
2,18500000,Mumbai,1150,Powai,2,1.0,9.0,9.0,9.0,9.0,...,9,9.0,9.0,9.0,9.0,9,9,9,NaN,16086.956522
3,6500000,Kolkata,1359,Sodepur,3,0.0,NaN,9.0,9.0,9.0,...,9,9.0,NaN,9.0,NaN,9,9,9,9.0,4782.928624
4,5800000,Mumbai,1000,Kamothe,2,1.0,1.0,1.0,0.0,0.0,...,1,0.0,0.0,0.0,0.0,0,0,0,0.0,5800.000000


In [10]:
df_summary = train_df.groupby(['city','Location'])[['price_per_foot^2']].mean()

In [11]:
df_summary.shape

(1446, 1)

In [12]:
train_df['Location'].nunique()

1437

In [13]:
train_df.shape

(20974, 42)

In [14]:
# creamos un var donde eliminamos espacios y digitos y pasamos a minusculas location
train_df['Standard_Location'] = train_df['Location'].str.lower().str.replace(' ','').str.replace(r'\d+','', regex=True)

In [15]:
train_df['Standard_Location'] = train_df['Standard_Location'].str.replace('sector','')

In [16]:
train_df['Standard_Location'].nunique()

1330

In [17]:
df_summary_2 = train_df.groupby(['city','Standard_Location'])[['price_per_foot^2']].mean()

In [18]:
city_location_dict = df_summary_2.to_dict()

In [19]:
city_location_dict = city_location_dict['price_per_foot^2']

In [20]:
city_location_dict['Bangalore', 'abbigere']

4639.438139601806

In [43]:
def predict_base_model(data):
    result = data.apply(
        func=lambda x: city_location_dict[x['city'], x['Standard_Location']]*x['Area'], 
        axis=1)
    return result.values

In [44]:
def extend_features(data):
    data.loc[:,'Standard_Location'] = ( data['Location'].
                                       str.lower().
                                       str.replace(' ','').
                                       str.replace(r'\d+','', regex=True).
                                       str.replace('sector','') )
    return data

In [45]:
train_df_eval = train_df.iloc[:2]
train_df_eval_extent = extend_features(train_df.iloc[:2])
predict_base_model(train_df_eval_extent)

array([25000000.        ,  6565374.55745636])

In [23]:
train_df_eval['Price']

0    45000000
1     7504000
Name: Price, dtype: int64

In [48]:
TEST_DATA = "data/preprocessed/test_data.csv"
test_data = pd.read_csv(TEST_DATA)
test_data_extent = extend_features(test_data)
predict_base_model(test_data_extent)


KeyError: ('Chennai', nan)

In [28]:
df_summary_city = train_df.groupby(['city'])[['price_per_foot^2']].mean()

In [29]:
df_summary_city

,price_per_foot^2
city,
Bangalore,6742.213920
Chennai,7093.361928
Delhi,12300.821212
Hyderabad,5518.984259
Kolkata,6881.439671
Mumbai,16038.331213


In [30]:
city_average_dict = df_summary_city.to_dict()['price_per_foot^2']

In [31]:
city_average_dict

{'Bangalore': 6742.21391989769,
 'Chennai': 7093.361927758863,
 'Delhi': 12300.821211920016,
 'Hyderabad': 5518.984258966283,
 'Kolkata': 6881.439670867204,
 'Mumbai': 16038.331212701598}

In [50]:
def predict_robust_base_model(data):
    def calculate_price(x):
        city_average = city_average_dict[x['city']]
        p_p_f = city_location_dict.get((x['city'], x['Standard_Location']),city_average)
        return p_p_f*x['Area']
    
    result = data.apply(
        func=calculate_price, 
        axis=1)
    return result.values

In [51]:
predict_robust_base_model(test_data_extent)

array([ 6803694.25552148, 19583827.69798527,  9580392.04268764, ...,
       12591464.68895297,  8162619.85444293,  2319062.54487435])

In [52]:
from sklearn.metrics import ( root_mean_squared_error, 
                             mean_absolute_error, 
                             mean_absolute_percentage_error )

In [53]:
y_true = test_data['Price'].values
y_pred = predict_robust_base_model(test_data_extent)
rmse = root_mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
print(f"rmse: {rmse}")
print(f"mae: {mae}")
print(f"mape: {mape}")

rmse: 21541646.873990554
mae: 7000403.157802502
mape: 0.8245420444659641
